# Introduction to Secure Multiparty Computation (SMPC)

This is the first notebook that will explain the basics of multiparty computation. In essence multiparty computation tries to outsource computations to different parties. Now, this per se is easy right? If I want a certain calculation to be performed in a remote computer I just send the data, the function to be calculated and that's it. The problem arises when one wants to outsource a computation without revealing the data, just the function. Secure Multiparty Computation (SMPC) is a set of techniques that allows us to compute a function without explicitely knowing the inputs.


## Yao's Millionaire problem

The typical application example of SMPC is the Yao's millionaire problem: Lets assume there are two millionaires, named Alice and Bob. Alice and Bob want to know who is richer without revealing how much money the other one has. This is what is known as the millionaires problem, originally introduced by [Andrew Yao](https://en.wikipedia.org/wiki/Andrew_Yao) in his famous [paper](https://ieeexplore.ieee.org/document/4568388). In essence they just want to know a bit of information, who's richer but not by which ammount. 

Now, let's assume we are an observer and we know Alice's and Bob net worth in the bank account

In [1]:
Alice_Wealth = 10
Bob_Wealth = 5
n = 20 #values to compute, must be larger than max(Alice_Wealth, Bob_Wealth)

Let's begin the protocol with Alice generating private and public keys for RSA protocol.

In [2]:
from crypt import RSAKeyGenerator
from random import seed

seed(3)
bits = 32
PublicKey, PrivateKey = RSAKeyGenerator(bits)

assert(PublicKey[0] == PrivateKey[0])
N = PublicKey[0]
e = PublicKey[1]
d = PrivateKey[1]

print(f"(N, e, d) = {N, e, d}")

(N, e, d) = (11498092178201763539, 2201574407153252135, 3293154980733467039)


Bob takes the values of the public key generated by Alice and chooses a random number U in N

In [3]:
from crypt import fastPowering
from random import randrange
from crypt import RSAEncrypt

U = randrange(2, 1<<bits)
C = RSAEncrypt(U, PublicKey) #encrypted U using e public key

print(f"Bob has chosen U={U} at random, whose encryption is C={C}")

Bob has chosen U=1407832020 at random, whose encryption is C=107670881904548396


Now Bob has to send to Alice a value Y1=C-BobWealth-1

In [4]:
y1 = C-Bob_Wealth+1
print(f"y1 = {y1}")

y1 = 107670881904548392


Alice calculates the RSA encryption of Y1, Y2... Yn.

In [5]:
from crypt import RSADecrypt

c = []
y = []

for x in range(n):
    c.append(y1+x-1)
    y.append(RSADecrypt(c[-1], PrivateKey))
    
    print(f"C-BobWealth+{x} = {c[-1]}, y{x}={y[-1]}")

C-BobWealth+0 = 107670881904548391, y0=2007605923776785506
C-BobWealth+1 = 107670881904548392, y1=4847244302080251977
C-BobWealth+2 = 107670881904548393, y2=10076270188377518848
C-BobWealth+3 = 107670881904548394, y3=10450182882653886449
C-BobWealth+4 = 107670881904548395, y4=88694198392758967
C-BobWealth+5 = 107670881904548396, y5=1407832020
C-BobWealth+6 = 107670881904548397, y6=5768553786887507302
C-BobWealth+7 = 107670881904548398, y7=6266906742779619174
C-BobWealth+8 = 107670881904548399, y8=7977301828422126600
C-BobWealth+9 = 107670881904548400, y9=11285198026163562562
C-BobWealth+10 = 107670881904548401, y10=9325124103173458571
C-BobWealth+11 = 107670881904548402, y11=3241550814594030995
C-BobWealth+12 = 107670881904548403, y12=9733075721223803821
C-BobWealth+13 = 107670881904548404, y13=9830506726019407413
C-BobWealth+14 = 107670881904548405, y14=691144743713013768
C-BobWealth+15 = 107670881904548406, y15=4224427468353387420
C-BobWealth+16 = 107670881904548407, y16=500009264981

Alice has to choose a random prime p in the range bits/2. This has to be done in a way that for when we apply the modulo $p$ to the values of $y$ do not differ less than 2. We will call $z$ to this new list

In [6]:
from crypt import RandomPrime

while True:
    p = RandomPrime(bits//2, 40)
    
    z = []
    for i, yx in enumerate(y):
        z.append(yx%p)
        
    z_sorted = z.copy()
    z_sorted.sort()
    min_diff = min([j-i for i, j in zip(z_sorted[:-1], z_sorted[1:])])
    
    if min_diff>1:
        break
    
    
print(f"Alice has chosen p={p} secretly s.t the encrypted values do not differ less than 2.")
print("These are the values of z:")
for i, elem in enumerate(z):
    print(f"z{i}={elem}")

Alice has chosen p=61291 secretly s.t the encrypted values do not differ less than 2.
These are the values of z:
z0=14956
z1=11716
z2=37364
z3=51936
z4=40926
z5=39041
z6=23941
z7=2965
z8=31163
z9=1775
z10=46422
z11=8799
z12=42029
z13=34450
z14=13873
z15=20244
z16=3119
z17=48342
z18=42236
z19=28387


Now with this secret value p prime she calculates yn (mod p)

Now she wants to send z to Bob but adding 1 from the value she has to the end:

In [7]:
print(f"Nottice that since worth of Alice is {Alice_Wealth} their indexes start to change from this index onwards")

Nottice that since worth of Alice is 10 their indexes start to change from this index onwards


Alice now sends the values of $zp$ to Bob and the prime number p. Now Bob looks at the index of his worth value and calculates G = U (mod p)

In [8]:
G = U%p
lookup_value = z[Bob_Wealth]
print(f"z{Bob_Wealth}={lookup_value}")
print(f"G={G}")

z5=39041
G=39041


In [9]:
if lookup_value == G:
    print("Alice worth is equal or greater to Bob's")
else:
    print("Alice worth is smaller to Bob's")

Alice worth is equal or greater to Bob's
